02/Jan/2024

# thining + outlier removing

## Thinning 

I use laspy.
Use the first method in the book, randomly. Homework requirement 0.25. Setting 0.25 means retaining 25% of the points.

## Outlier removing

Outlier I use scipy.

Originally I used sklearn, but the teaching assistant said it was not suitable (there are public questions on discord)

1. Use the fourth method in the book - knn distance.
2. Set the parameter to 3 and use the median. 
3. Corrected the problem why the cloudcompare is so strange when checking with Xiao Luo at 14:00 02/Jan/2024...because when I process it, I convert it to np.array and write it back to the laz file, which will lose some information. Such as color, Deviation Reflectance Amplitude PointSourceld UserData ScanAngleRank EdgeOf FlightLine ScanDirecitonFlag NumberOfReturns GpsTime Classification and so on. The program code has been modified to retain the message. I threw it into Cloud Compare myself and there should be no problem.

## Input data

1. tile_4_600.laz :  600x600m (the one with buffer) need thining+outlier removing
2. tile_4_500.laz :  500x500m just need to remove the outlier, because step4 don't need to thining at the begining.


# Thining


In [26]:
# Thining

import laspy
import numpy as np

def thin_laz_file(input_file, output_file, keep_fraction=0.25): # NOTICE HERE!! 0.25 assingment required 
    # read LAZ file
    with laspy.open(input_file) as file:
        las = file.read()

    # get how many numbers
    total_points = las.header.point_count

    # Randomly select points to keep 
    # 11.1 thinning method 1 random
    chosen_indices = np.random.choice(total_points, int(total_points * keep_fraction), replace=False)

    # keep the points
    thinned_las = las[chosen_indices]

    # write into new file
    with laspy.open(output_file, mode="w", header=las.header) as outfile:
        outfile.write_points(thinned_las.points)

    print(f"Thinning complete. Saved to {output_file}")

# how to use the function
input_laz_path = "tile_4_600.laz"  # the file's path, this is the one which is cropped
output_laz_path = "600_thinned_025.laz"  # name of the output file

thin_laz_file(input_laz_path, output_laz_path)

Thinning complete. Saved to 600_thinned_025.laz


In [27]:
# check the size
def read_las(file_path):
    """
    read files
    """
    return laspy.read(file_path)

def get_bounds(las):
    """
    get bounds
    """
    min_x, max_x = las.header.x_min, las.header.x_max
    min_y, max_y = las.header.y_min, las.header.y_max
    return min_x, max_x, min_y, max_y

file_path = "600_thinned_025.laz"
print(get_bounds(read_las("tile_4_600.laz")))
print(get_bounds(read_las("600_thinned_025.laz")))

(188415.0, 189015.0, 311750.0, 312350.0)
(188415.0, 189015.0, 311750.0, 312350.0)


# Outliers removing

In [28]:
# Outliers removing

import laspy
import numpy as np
from scipy.spatial import KDTree

def remove_outliers_laz(file_path, k=3):
    # read laz 
    las = laspy.read(file_path)

    points = np.vstack((las.x, las.y, las.z)).transpose()

    # create KDTree
    tree = KDTree(points)

    # Calculate the k nearest neighbor distances for each point
    distances, _ = tree.query(points, k=k+1)  # k+1 Because the nearest point is itself
    median_distances = np.median(distances[:, 1:], axis=1)  # Calculate median distance

    # Determine threshold for outliers
    threshold = np.mean(median_distances) + 2 * np.std(median_distances)  
 
    # Filter out outliers. Values greater than the threshold will be deleted.
    mask = median_distances < threshold

    # Create filtered LAS file to save 
    filtered_las = laspy.create(point_format=las.header.point_format, file_version=las.header.version)
    
    """TThe following is to retain the attributes of laz data"""

    # Copy all the attributes in the original las, just the long list above
    for name in las.point_format.dimension_names:
        if name in las.point_format.dimension_names:
            data = getattr(las, name)
            setattr(filtered_las, name, data[mask])

    # Preserve scale factors and offsets
    # If you don’t do this step, its xyz will run away.
    filtered_las.header.scale = las.header.scale
    filtered_las.header.offset = las.header.offset

    # Save the processed file
    output_path = file_path.replace('.laz', '_filtered.laz')
    filtered_las.write(output_path)
    print(f"Filtered LAS file saved as: {output_path}")

# how to use
remove_outliers_laz("tile_4_500.laz", k=3) # This is for step 4
remove_outliers_laz("600_thinned_025.laz", k=3) # This is the one after thining and will be use in step 3

Filtered LAS file saved as: tile_4_500_filtered.laz
Filtered LAS file saved as: 600_thinned_025_filtered.laz


# Be careful of thining

Sometimes thining will delete boundary points. We have to make sure that the coordinates of the lower left and upper right points maintain our size. So if this situation occurs, use the following code to do thining

In [29]:
import laspy
import numpy as np

def thin_laz_file(input_file, output_file, keep_fraction=0.25):
    # Read LAZ file
    with laspy.open(input_file) as file:
        las = file.read()

    # Get total number of points
    total_points = las.header.point_count

    # Randomly select points to keep
    chosen_indices = np.random.choice(total_points, int(total_points * keep_fraction), replace=False)

    # Ensure boundary points are included
    boundary_indices = get_boundary_indices(las)
    chosen_indices = np.unique(np.concatenate((chosen_indices, boundary_indices)))

    # Keep the points
    thinned_las = las[chosen_indices]

    # Write to new file
    with laspy.open(output_file, mode="w", header=las.header) as outfile:
        outfile.write_points(thinned_las.points)

    print(f"Thinning complete. Saved to {output_file}")

def get_boundary_indices(las):
    xmin, xmax, ymin, ymax = las.header.min[0], las.header.max[0], las.header.min[1], las.header.max[1]

    # Find indices of points that are on the boundary
    boundary_indices = np.where(
        (las.x == xmin) | (las.x == xmax) | (las.y == ymin) | (las.y == ymax)
    )[0]

    return boundary_indices

    """
input_laz_path = "tile_4_600.laz"  
output_laz_path = "600_thinned_01.laz"  

thin_laz_file(input_laz_path, output_laz_path)
    """

# Output files

1. 600_thinned_025.laz  : 600x600m 0.25 thining
2. tile_4_500_filtered.laz  : 500x500m outlier removing and without thining
3. 600_thinned_025_filtered.laz : 600x600m 0.25 thining+outlier removing

## Just the test. Ignore it


In [30]:
import laspy

def read_laz_properties(file_path):

    laz_file = laspy.read(file_path)

    # Get the required properties
    properties = {
        "x":laz_file.x,
        "y":laz_file.y,
        "z":laz_file.z,
        "colors": laz_file.color if hasattr(laz_file, 'color') else 'No color data',
        "intensity": laz_file.intensity,
        "return_number": laz_file.return_number,
        "number_of_returns": laz_file.number_of_returns,
        "scan_direction_flag": laz_file.scan_direction_flag,
        "edge_of_flight_line": laz_file.edge_of_flight_line,
        "classification": laz_file.classification,
       # "scan_angle_rank": laz_file.scan_angle_rank,
        "user_data": laz_file.user_data,
        "point_source_id": laz_file.point_source_id,
        "gps_time": laz_file.gps_time,
        "amplitude": laz_file.amplitude if hasattr(laz_file, 'amplitude') else 'No amplitude data',
        "deviation": laz_file.deviation if hasattr(laz_file, 'deviation') else 'No deviation data'
    }

    return properties




In [32]:
file_path = "600_thinned_025.laz"  # crop + thining one
properties = read_laz_properties(file_path)
properties

{'x': <ScaledArrayView([188684.583 188728.947 188588.742 ... 188427.602 188467.597 188660.982])>,
 'y': <ScaledArrayView([312241.676 312121.377 312322.708 ... 312149.996 312037.754 312094.263])>,
 'z': <ScaledArrayView([118.058 112.656 120.47  ... 130.939 131.184 113.124])>,
 'colors': 'No color data',
 'intensity': array([1290, 1259, 1258, ..., 1394,  707, 1299], dtype=uint16),
 'return_number': <SubFieldView([1 1 1 ... 1 1 1])>,
 'number_of_returns': <SubFieldView([1 1 1 ... 1 2 1])>,
 'scan_direction_flag': <SubFieldView([0 1 0 ... 0 0 0])>,
 'edge_of_flight_line': <SubFieldView([0 0 0 ... 0 0 0])>,
 'classification': array([6, 2, 2, ..., 2, 1, 2], dtype=uint8),
 'user_data': array([6, 7, 6, ..., 6, 6, 6], dtype=uint8),
 'point_source_id': array([152, 152, 152, ..., 152, 152, 152], dtype=uint16),
 'gps_time': array([2.97679757e+08, 2.97679757e+08, 2.97679758e+08, ...,
        2.97679762e+08, 2.97679762e+08, 2.97679759e+08]),
 'amplitude': 'No amplitude data',
 'deviation': 'No devia

In [33]:
file_path = "600_thinned_025_filtered.laz" # crop + thining + outlier
properties = read_laz_properties(file_path)
properties

{'x': <ScaledArrayView([188684.583 188728.947 188588.742 ... 188427.602 188467.597 188660.982])>,
 'y': <ScaledArrayView([312241.676 312121.377 312322.708 ... 312149.996 312037.754 312094.263])>,
 'z': <ScaledArrayView([118.058 112.656 120.47  ... 130.939 131.184 113.124])>,
 'colors': 'No color data',
 'intensity': array([1290, 1259, 1258, ..., 1394,  707, 1299], dtype=uint16),
 'return_number': <SubFieldView([1 1 1 ... 1 1 1])>,
 'number_of_returns': <SubFieldView([1 1 1 ... 1 2 1])>,
 'scan_direction_flag': <SubFieldView([0 1 0 ... 0 0 0])>,
 'edge_of_flight_line': <SubFieldView([0 0 0 ... 0 0 0])>,
 'classification': array([6, 2, 2, ..., 2, 1, 2], dtype=uint8),
 'user_data': array([6, 7, 6, ..., 6, 6, 6], dtype=uint8),
 'point_source_id': array([152, 152, 152, ..., 152, 152, 152], dtype=uint16),
 'gps_time': array([2.97679757e+08, 2.97679757e+08, 2.97679758e+08, ...,
        2.97679762e+08, 2.97679762e+08, 2.97679759e+08]),
 'amplitude': 'No amplitude data',
 'deviation': 'No devia